### 为了缓解技术支持人力不足与课堂网络工单不断增长的矛盾，拟通过算法模型预测工单对应的课堂是否确实需要技术支持。
### 上一版算法以轻微抖动课堂/严重卡顿课堂作为正反例对比建模，旨在在工单提交时预测网络工单的关闭原因，避免对轻微抖动课堂的工单浪费人力，然而建模效果很不理想，现结合业务背景将算法建模的目的修正为：区分特定课堂是否需要提网络工单，即以正常课堂/严重卡顿课堂作为正反例对比建模。
### 此次建模共采用了1296条样本（正负样本各半），14个监控指标延伸出的560维特征，试验了三种降维方式，logistics回归、随机森林两种机器学习模型，最终模型效果达到精确率93.8%，召回率76.9%（即预测为严重卡顿课堂的工单里有93.8%确实是严重卡顿课堂，所有真正严重卡顿的课堂里有76.9%被成功预测）
### 模型使用建议：通过此模型对所有课堂网络工单进行实时标记（正常/异常），请技术支持小伙伴优先处理有异常标记的工单；持续跟进模型效果；待积累一定的数据量后进行算法迭代。

### 模型抽取（重构？）

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation, metrics  
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

df=pd.read_csv("E:/Jupyter/3.15junchen_hire_project/data.csv")

In [ ]:
# 删除干扰项
del_columns = df.columns.str.contains('sentFrameRate') + df.columns.str.contains('sentBitrate') +df.columns.str.contains('_lag_') 
df = df.loc[:,~del_columns]
# 填补缺失值并删除有大量异常值的行，只要少量异常值的行将异常值替换为该列均值
df.fillna(-999,inplace=True)
df3=df[~df.isin([-999])].dropna(thresh = 521)

for column in list(df3.columns[df3.isnull().sum() > 0]):
    mean_val = df3[column].mean()
    df3[column].fillna(mean_val, inplace=True)
    
# 删掉type=1的轻微抖动课堂
df4 = df3[df3['type'] != 1]    

# 划分正负样本，并将正例lable改为1
df_F = df4[df4['type'] == 0].sample(frac=1)
df_T = df4[df4['type'] == 2]
df_T['type'] = df_T['type'].map(lambda x: x-1)

# 抽取和正例同样数量的负例作为建模数据集（df_648）
df_F_648=df_F[0:648]
df_648 = pd.concat([df_F_648,df_T],axis=0)

In [ ]:
# 取每个特征的lead_1-lead-5通过随机森林建模
lead5_columns = df_648.columns.str.contains('type')+df_648.columns.str.contains(r"lead_1$",regex=True) +df_648.columns.str.contains(r"lead_2$",regex=True)+df_648.columns.str.contains(r"lead_3$",regex=True)+df_648.columns.str.contains(r"lead_4$",regex=True)+df_648.columns.str.contains(r"lead_5$",regex=True) 
df_lead5 = df_648.loc[:,lead5_columns]

# 划分训练集和测试集
x_columns_lead5 = [x for x in df_lead5.columns if x not in ['type']]
X_lead5 = df_lead5[x_columns_lead5]
y_lead5 = df_lead5['type']

X_train_lead5, X_test_lead5, y_train_lead5, y_test_lead5 = train_test_split(X_lead5,y_lead5,test_size=0.2, random_state=0)

In [ ]:
# 最终模型——精确率93.85%，召回率76.98%
forest = RandomForestClassifier(n_estimators= 1000,max_depth=8,max_features=8,oob_score=True, random_state=10,n_jobs=-1)
forest.fit(X_train_lead5,y_train_lead5)
print('袋外得分：',forest.oob_score_) 

y_predict_lead5=forest.predict(X_test_lead5)
recall_lead5 = recall_score(y_test_lead5,y_predict_lead5)
print('召回率：',recall_lead5)
precision_lead5  = precision_score(y_test_lead5,y_predict_lead5)
print('精确率：',precision_lead5 )

In [ ]:
# 特征重要性排序
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

%matplotlib inline
feat_labels=X_train_lead5.columns
for f in range(X_train_lead5.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importances')
plt.bar(range(X_train_lead5.shape[1]), 
        importances[indices],
        color='lightblue', 
        align='center')

plt.xticks(range(X_train_lead5.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train_lead5.shape[1]])
plt.tight_layout()
#plt.savefig('./random_forest.png', dpi=300)
plt.show()

### 详细建模过程

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation, metrics  
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
df=pd.read_csv("E:/Jupyter/3.15junchen_hire_project/data.csv")

In [ ]:
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

### 简单看下数

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df['type'].value_counts()

### 删掉sentFrameRate和sentBitrate两个干扰项，且由于提交工单时后续情况尚未发生，去掉所有带\_lag\_的特征

In [ ]:
del_columns = df.columns.str.contains('sentFrameRate') + df.columns.str.contains('sentBitrate') +df.columns.str.contains('_lag_') 

In [ ]:
df = df.loc[:,~del_columns]

In [ ]:
df.info()

### 样本量5000，特征561——考虑降维，此前先做特征工程

In [ ]:
df.describe()

In [ ]:
df.count()

### 有缺失值，删光有缺失值的行看看还剩多少

In [ ]:
df1=df.dropna(axis=0, how='any')

In [ ]:
df1.count()

### 还剩4977样本，影响很小，缺失值简单处理就好——比如补-999

In [ ]:
df.fillna(-999,inplace=True)

### 看看删光异常值影响多大

In [ ]:
df2 = df[~df.isin([-999])].dropna(axis=0)

In [ ]:
df2.count()

In [ ]:
df2['type'].value_counts()

### 样本量太少，删了有点可惜，看了下-999主要在和CPU、内存有关的几个指标上，打算删掉有40个以上-999的行，其余-999按列补上均值

In [ ]:
df3=df[~df.isin([-999])].dropna(thresh = 521)

In [ ]:
df3.count()

In [ ]:
for column in list(df3.columns[df3.isnull().sum() > 0]):
    mean_val = df3[column].mean()
    df3[column].fillna(mean_val, inplace=True)

In [ ]:
df3['type'].value_counts()

### 数据不平衡，由于业务场景变更为区分是否要提工单，删掉type=1的轻微抖动样本（轻微抖动课堂本身不应该提工单，但这部门课堂总体确实要比正常课堂要差，作为负例会对模型造成干扰，且作为负例的正常课堂样本量远大于作为正例的严重抖动课堂，不差type=1课堂的这点信息增益）

In [ ]:
df4 = df3[df3['type'] != 1]

In [ ]:
df4['type'].value_counts()

### 划分正负样本

In [ ]:
df_F = df4[df4['type'] == 0].sample(frac=1)
df_T = df4[df4['type'] == 2]
df_T['type'] = df_T['type'].map(lambda x: x-1)

### 以正负样本1:1，PCA降维，logistic回归做一版基线模型

In [ ]:
# 随机抽648个type=0的样本
df_F_648=df_F[0:648]
df_648 = pd.concat([df_F_648,df_T],axis=0)

In [ ]:
df_648['type'].value_counts()

### 划分训练集合测试集

In [ ]:
x_columns = [x for x in df_648.columns if x not in ['type']]
X = df_648[x_columns]
y = df_648['type']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

### 标准化数据（PCA需要，单做随机森林等树算法则不用）

In [ ]:
scaler = StandardScaler()
# 只能在训练集上折腾，测试集是黑盒
scaler.fit(X_train)

X_train_scal = scaler.transform(X_train)
X_test_scal = scaler.transform(X_test)

### 采用PCA降维，保留85％的方差

In [ ]:
pca_p85 = PCA(0.85)
X_train_scal_pca_p85 = pca_p85.fit_transform(X_train_scal)
print(pca_p85.explained_variance_ratio_)

In [ ]:
pca_p80 = PCA(0.8)
X_train_scal_pca_p80 = pca_p80.fit_transform(X_train_scal)
print(pca_p80.explained_variance_ratio_)

In [ ]:
pca_85 = PCA(60)
X_train_scal_pca_85 = pca_85.fit_transform(X_train_scal)
X_test_scal_pca_85 = pca_85.fit_transform(X_test_scal)

In [ ]:
classifier=LogisticRegression()

scores=cross_val_score(classifier,X_train_scal_pca_85,y_train,cv=5)
print('准确率',np.mean(scores),scores)
precisions = cross_val_score(classifier,X_train_scal_pca_85, y_train, cv=5, scoring='precision')
print(u'精确率：', np.mean(precisions), precisions)
recalls = cross_val_score(classifier, X_train_scal_pca_85, y_train, cv=5, scoring='recall')
print(u'召回率：', np.mean(recalls), recalls)

### 多次尝试，发现样本1:1时pca保留0.85比0.8、0.9稍好，按pca0.85，尝试加大样本量看下模型效果，发现还是1:1最好

### 尝试每8列的均值/标准差的降维方式（降到70维）

In [ ]:
X_70mean=X.groupby(np.arange(len(X.columns))//8, axis=1).mean()

In [ ]:
X_70std=X.groupby(np.arange(len(X.columns))//8, axis=1).std()   #貌似该用总体标准差，pandas写总体标准差有点麻烦，先用样本标准差看看效果

In [ ]:
# X_70std中有0
X_70std_tem=X_70std[~X_70std.isin([0])]
X_70std_tem.fillna(0.1,inplace=True)

In [ ]:
X_70=X_70mean/X_70std_tem

In [ ]:
scaler_X_70 = StandardScaler()
scaler_X_70.fit(X_70)
X_70scal = scaler_X_70.transform(X_70)

In [ ]:
classifier_X_70=LogisticRegression()

scores_X_70=cross_val_score(classifier_X_70,X_70scal,y,cv=5)
print('准确率',np.mean(scores_X_70),scores_X_70)
precisions_X_70 = cross_val_score(classifier_X_70,X_70scal, y, cv=5, scoring='precision')
print(u'精确率：', np.mean(precisions_X_70), precisions_X_70)
recalls_X_70 = cross_val_score(classifier_X_70,X_70scal, y, cv=5, scoring='recall')
print(u'召回率：', np.mean(recalls_X_70), recalls_X_70)

### 这里均值/标准差的降维方式效果远不如PCA，下面尝试直接取lead_1-lead-5，用随机森林建模并看下特征重要性排序

In [ ]:
lead5_columns = df_648.columns.str.contains('type')+df_648.columns.str.contains(r"lead_1$",regex=True) +df_648.columns.str.contains(r"lead_2$",regex=True)+df_648.columns.str.contains(r"lead_3$",regex=True)+df_648.columns.str.contains(r"lead_4$",regex=True)+df_648.columns.str.contains(r"lead_5$",regex=True) 

In [ ]:
df_lead5 = df_648.loc[:,lead5_columns]

In [ ]:
x_columns_lead5 = [x for x in df_lead5.columns if x not in ['type']]
X_lead5 = df_lead5[x_columns_lead5]
y_lead5 = df_lead5['type']

X_train_lead5, X_test_lead5, y_train_lead5, y_test_lead5 = train_test_split(X_lead5,y_lead5,test_size=0.2, random_state=0)

In [ ]:
forest = RandomForestClassifier(n_jobs=-1)
forest.fit(X_lead5, y_lead5)

importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

In [ ]:
%matplotlib inline

feat_labels=X_lead5.columns

for f in range(X_lead5.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importances')
plt.bar(range(X_lead5.shape[1]), 
        importances[indices],
        color='lightblue', 
        align='center')

plt.xticks(range(X_lead5.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_lead5.shape[1]])
plt.tight_layout()
#plt.savefig('./random_forest.png', dpi=300)
plt.show()

### 长尾效应非常明显，这里应该也有部分优化空间，还是先试下随机森林建模的效果

In [ ]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)  
rf0.fit(X_train_lead5,y_train_lead5) 
print(rf0.oob_score_) 
y_predprob_lead5 = rf0.predict_proba(X_test_lead5)[:,1]  
print("AUC Score: %f" % metrics.roc_auc_score(y_test_lead5,y_predprob_lead5)) 

In [ ]:
y_predict_lead5=rf0.predict(X_test_lead5)

recall_rf0 = recall_score(y_test_lead5,y_predict_lead5)
print('召回率：', recall_rf0)
precision_rf0 = precision_score(y_test_lead5,y_predict_lead5)
print('精确率：', precision_rf0)

### 随机森林效果好，这个小模型用随机森林线上也能支持，暂定用随机森林，开始调参

In [ ]:
param_test1= {'n_estimators': [10, 100, 500, 1000, 1500]}  
gsearch1= GridSearchCV(estimator = RandomForestClassifier(random_state=10),param_grid =param_test1, scoring='roc_auc',cv=3)  
gsearch1.fit(X_train_lead5,y_train_lead5)  
gsearch1.grid_scores_,gsearch1.best_params_, gsearch1.best_score_  

In [ ]:
rf1 = RandomForestClassifier(n_estimators= 1000,oob_score=True, random_state=10,n_jobs=-1)
rf1.fit(X_train_lead5,y_train_lead5)
print(rf1.oob_score_) 

In [ ]:
param_test2= {'n_estimators': range(600,1400,100)}  
gsearch2= GridSearchCV(estimator = RandomForestClassifier(random_state=10),param_grid =param_test2, scoring='roc_auc',cv=3,n_jobs=-1)  
gsearch2.fit(X_train_lead5,y_train_lead5)  
gsearch2.grid_scores_,gsearch2.best_params_, gsearch2.best_score_ 

In [ ]:
param_test3= {'max_depth':range(2,20,2)} 
gsearch3= GridSearchCV(estimator = RandomForestClassifier(n_estimators=1000 ,oob_score=True,random_state=10),  
                      param_grid = param_test3,scoring='roc_auc',cv=3,n_jobs=-1)  
gsearch3.fit(X_train_lead5,y_train_lead5)  
gsearch3.grid_scores_,gsearch3.best_params_,gsearch3.best_score_  

In [ ]:
param_test4= {'max_features':range(2,35,3)}  
gsearch4= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 1000,max_depth=8,oob_score=True, random_state=10),  
                       param_grid = param_test4,scoring='roc_auc',cv=3,n_jobs=-1)  
gsearch4.fit(X_train_lead5,y_train_lead5)  
gsearch4.grid_scores_,gsearch4.best_params_, gsearch4.best_score_  

In [ ]:
rf2 = RandomForestClassifier(n_estimators= 1000,max_depth=8,max_features=8,oob_score=True, random_state=10,n_jobs=-1)
rf2.fit(X_train_lead5,y_train_lead5)
print(rf2.oob_score_) 

In [ ]:
param_test5= {'min_samples_split':[2,4,10,20,40], 'min_samples_leaf':[1,2,4,10,20]}  
gsearch5= GridSearchCV(estimator = RandomForestClassifier(n_estimators= 1000,max_depth=8,max_features=8,oob_score=True, random_state=10),  
                       param_grid = param_test5,scoring='roc_auc',cv=2,n_jobs=-1)  
gsearch5.fit(X_train_lead5,y_train_lead5)  
gsearch5.grid_scores_,gsearch5.best_params_,gsearch5.best_score_ 

In [ ]:
#召回率
y_predict_lead5=rf2.predict(X_test_lead5)
recall_lead5 = recall_score(y_test_lead5,y_predict_lead5)
print('召回率：',recall_lead5)

In [ ]:
precision_lead5  = precision_score(y_test_lead5,y_predict_lead5)
print('精确率：',precision_lead5 )

### 下一步可以根据特征重要性特征排序筛选一些重要特征，或把特征lead_1-lead_5调整为lead_1-lead_3等方式调整特征维度，建模与现有模型做对比，不过优化空间应该不大了

### 此模型最终的整体效果还行，但召回率（查全率）只有77%，并不是特别理想，如果需要达到很高的召回率（将牺牲更多精确率），可以通过如logistic回归的罚函数等方式实现